In [10]:
library(kgc)
library(dplyr)
library(tidyr)

In [11]:
network_metadata_path = "../../data/networks/networks_metadata.csv"

In [12]:
network_metadata = read.csv(network_metadata_path)
relevant_network_data = drop_na(network_metadata, any_of(c("Longitude", "Latitude")))
relevant_network_data = relevant_network_data[, c("network_type", "network_index", "Longitude", "Latitude")]

In [13]:
relevant_network_data = data.frame(relevant_network_data, rndCoord.lat = RoundCoordinates(relevant_network_data$Latitude), rndCoord.lon = RoundCoordinates(relevant_network_data$Longitude))

In [14]:
relevant_network_data = data.frame(relevant_network_data,ClimateZ=LookupCZ(relevant_network_data))

In [15]:
relevant_network_data <- data.frame(relevant_network_data, CZUncertainty(relevant_network_data))

In [16]:
relevant_network_data[relevant_network_data$ClimateZ == "Climate Zone info missing", c("ClimateZ")] = NA

In [17]:
# categorization is devised from http://koeppen-geiger.vu-wien.ac.at/present.htm
ClimateZ = c(equatorial = c("Af", "Am", "As", "Aw"),
             arid = c("BSh", "BSk", "BWh", "BWk"),
             warm_temprate = c("Cfc", "Cfb", "Csc", "Csb", "Cfa", "Csa", "Cwa", "Cwb", "Cwc"),
             boreal = c("Dwc", "Dwb", "Dsa", "Dsb", "Dfb", "Dfc", "Dsc", "Dwa", "Dfa", "Dwd", "Dfd"),
             polar = c("EF", "ET"))
clim_cat_data = data.frame(ClimateZ)
clim_cat_data["climate_category"] = rownames(clim_cat_data)
clim_cat_data$climate_category = gsub("[0-9]*","",clim_cat_data$climate_category)
rownames(clim_cat_data) = NULL

In [18]:
relevant_network_data <- merge(relevant_network_data, clim_cat_data, by.x = "ClimateZ", 
             by.y = "ClimateZ", all.x = TRUE, all.y = FALSE)

In [19]:
network_metadata = merge(network_metadata, relevant_network_data[, c("network_type", "network_index", "ClimateZ", "climate_category")], by.x = c("network_type", "network_index"), 
             by.y = c("network_type", "network_index"), all.x = TRUE, all.y = FALSE)

In [23]:
head(drop_na(network_metadata, any_of(c("Latitude")))[, c("network_type", "network_index", "Latitude", "Longitude", "climate_category")])

,network_type,network_index,Latitude,Longitude,climate_category
,<chr>,<int>,<dbl>,<dbl>,<chr>
1,binarized_weighted,13,48.40838,10.42863,warm_temprate
2,binarized_weighted,14,48.40838,10.42863,warm_temprate
3,binarized_weighted,15,48.40838,10.42863,warm_temprate
4,binarized_weighted,16,48.40838,10.42863,warm_temprate
5,binarized_weighted,17,48.40838,10.42863,warm_temprate
6,binarized_weighted,18,48.40838,10.42863,warm_temprate


In [25]:
unique(network_metadata$climate_category)

[1] NA              "warm_temprate" "boreal"        "polar"        
[5] "arid"          "equatorial"

In [33]:
debug = network_metadata[is.na(network_metadata$climate_category), c("EcoRegion_name", "ClimateZ")] # "Longitude", "Latitude", 
debug = unique(debug)
debug

,EcoRegion_name,ClimateZ
,<chr>,<fct>
1,,NA
203,mascarene forests,NA
347,azores temperate mixed forests,NA
501,galapagos islands xeric scrub,NA
506,nihonkai montane deciduous forests,NA
771,sulawesi lowland rain forests,NA
791,talamancan montane forests,NA
799,nansei islands subtropical evergreen forests,NA
855,celtic broadleaf forests,NA


In [24]:
write.csv(network_metadata, network_metadata_path)